### 第一个聊天机器人

In [1]:
# 简单进行一个机器翻译样例
from langchain.chat_models import ChatOpenAI # 导入ChatOpenAI类，以访问由vllm提供的OpenAI API
# 导入Messages类
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [14]:
# 创建一个ChatOpenAI对象
chat = ChatOpenAI(
    streaming=True,
    verbose=True,
    openai_api_key="none", # 这里由于我们使用的是vllm本地LLM的API，所以填写none
    openai_api_base="http://127.0.0.1:8000/v1",
    model_name="Qwen/Qwen1.5-4B-Chat",
    temperature=0
) 
chat.predict_messages([
    HumanMessage(
        content=("Translate this sentence from English to Chinese."
                 "I love programming.")
    )
])

AIMessage(content='我喜欢编程。')

### 提示词模板

In [15]:
# 使用MessagePromptTemplate来创建一个提示词模板
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
# 通过“留槽”的方式来创建一个模板
template = (
    "You are a helpful assistant that translates {input_language} to"
    "{output_language}"
)
# 创建系统提示词模板
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
# 创建用户输入提示词模板
human_template = "{text}"

human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
# 创建一个聊天提示词模板
chat_prompt = ChatPromptTemplate.from_messages(
    [
        system_message_prompt,
        human_message_prompt,
    ]
)
# Langchain将自动检索所有Messages中预留的格式化输出槽，因此我们只需要像设置参数一样把提示词输入进去即可了
chat_prompt.format_messages(
    input_language="English",
    output_language="Chinese",
    text="I love programming."
)

[SystemMessage(content='You are a helpful assistant that translates English toChinese'),
 HumanMessage(content='I love programming.')]

### 创建第一个链

In [20]:
# 导入LLMChain类
from langchain import LLMChain

# 定义一个链
chain = LLMChain(llm=chat, prompt=chat_prompt)
# 运行链
chain.run(
    input_language="English",
    output_language="Chinese",
    text="I love programming."
)

'我热爱编程。'

### 创建第一个智能体(Agent)

In [28]:
import os
os.environ['SERPAPI_API_KEY'] = "<YOUR_SERPAPI_KEY>"

In [26]:
# 为了确保搜索的可用性，请安装google-search-results和numexpr
!pip install google-search-results
!pip install numexpr

Looking in indexes: https://mirrors.ustc.edu.cn/pypi/web/simple
Looking in indexes: https://mirrors.ustc.edu.cn/pypi/web/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.5/377.5 kB 11.9 MB/s eta 0:00:00


In [29]:
# 加载智能体需要的工具
from langchain.agents import (
    load_tools,
    initialize_agent,
    AgentType,
)
# 加载工具
tools = load_tools(['serpapi','llm-math'],llm=chat)

# 初始化智能体
agent = initialize_agent(tools,chat,agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

# 运行智能体
agent.run(
    "What will be weather in Shenyang three days from now?"
)



> Entering new AgentExecutor chain...
Thought: I need to find out the weather in Shenyang three days from now. I can use the search engine to do this.
Action:
```
{
  "action": "Search",
  "action_input": "weather in Shenyang three days from now"
}
```
Observation: {'type': 'weather_result', 'temperature': '36', 'unit': 'Fahrenheit', 'precipitation': '0%', 'humidity': '25%', 'wind': '13 mph', 'location': 'Shenyang, Liaoning, China', 'date': 'Tuesday', 'weather': 'Mostly cloudy'}
Thought:I now know the final answer
Final Answer: According to the weather forecast, it will be mostly cloudy in Shenyang three days from now with a temperature of 36°F and a humidity of 25%. The wind speed is 13 mph.

> Finished chain.


'According to the weather forecast, it will be mostly cloudy in Shenyang three days from now with a temperature of 36°F and a humidity of 25%. The wind speed is 13 mph.'

### 记忆组件

In [30]:
# 导入对话链
from langchain.chains import ConversationChain
# 导入MessagePlaceHolder
from langchain.prompts import MessagesPlaceholder
# 导入记忆组件
from langchain.memory import ConversationBufferMemory

# 创建新的Message
prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(
        """
        The following is a friendly conversation between a human and an AI.
        The AI istalkative and provides lots of specific details from its context.
        If the AI does not know the answer to a question,it truthfully says it does not know.
        """
    ),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}"),
])

In [31]:
# 创建一个对话记忆组件
memory = ConversationBufferMemory(return_messages=True)

In [32]:
# 创建一个对话链
conversation = ConversationChain(memory=memory,prompt=prompt,llm=chat)

In [33]:
conversation.predict(input="你好，我是大霖！")

'你好，大霖！很高兴能和你聊天。有什么我可以帮助你的吗？'